Source: https://github.com/pytorch/examples/tree/master/reinforcement_learning and https://github.com/pytorch/tutorials/blob/master/Reinforcement%20(Q-)Learning%20with%20PyTorch.ipynb.

# Policy Gradient Methods

## REINFORCE

In [1]:
# import libraries
import numpy as np
from itertools import count # useful for counting loop
import argparse
import gym
from IPython.core.debugger import Tracer

In [2]:
# PyTorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable

In [3]:
parser = {
    "gamma": 0.99, # discount
    "seed": 543, # seed for reprocedure
    "render": False, # draw or not
    "log_interval": 10, # when to print results
}
args = argparse.Namespace(**parser)

In [4]:
# init the environment and seeds
env = gym.make("CartPole-v0")
env.seed(args.seed)
torch.manual_seed(args.seed)

[2017-02-21 22:14:55,190] Making new env: CartPole-v0


In [5]:
# define the model
class Policy(nn.Module):
    """State is 4-dimensional vector and there are 2 actions."""
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.affine2 = nn.Linear(128, 2)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        # return the score of each action (left or right)
        # state-action value / Q-learning
        x = F.relu(self.affine1(x))
        action_scores = self.affine2(x)
        return F.softmax(action_scores)

In [6]:
# init model and define optimizer
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [7]:
# utilities
def select_action(state):
    # in PyTorch, we always use minibatch, so use unsqueeze to make
    # fake batch 1 dimension
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = model(Variable(state))
    action = probs.multinomial()
    model.saved_actions.append(action)
    return action.data

def finish_episode():
    R = 0
    saved_actions = model.saved_actions
    rewards = []
    for r in model.rewards[::-1]:
        R = r + args.gamma * R
        rewards.insert(0, R)
    rewards = torch.Tensor(rewards)
    rewards = (rewards - rewards.mean()) / rewards.std()
    # the following is REINFORCE update, using StochasticVariable node
    # see the paper https://arxiv.org/abs/1506.05254 for more details
    for action, r in zip(model.saved_actions, rewards):
        action.reinforce(r) # PyTorch implementation for Variable
    optimizer.zero_grad()
    autograd.backward(model.saved_actions, [None for _ in model.saved_actions])
    optimizer.step() # update parameters
    del model.rewards[:] # reset to empty list
    del model.saved_actions[:]

In [8]:
# training, using running_reward is criterion for stop
running_reward = 10
for i_episode in count(1):
    state = env.reset()
    for t in range(10000): # Don't infinite loop while learning
        action = select_action(state)
        state, reward, done, _ = env.step(action[0,0])
        if args.render:
            env.render()
        model.rewards.append(reward)
        if done:
            break

    running_reward = running_reward * 0.99 + t * 0.01
    finish_episode()
    if i_episode % args.log_interval == 0:
        print("Episode {}\tLast length: {:5d}\tAverage length: {:.2f}"
              .format(i_episode, t, running_reward))
    if running_reward > 200:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!"
              .format(running_reward, t))
        break

Episode 10	Last length:    13	Average length: 10.64
Episode 20	Last length:    24	Average length: 11.37
Episode 30	Last length:   115	Average length: 15.63
Episode 40	Last length:    17	Average length: 19.16
Episode 50	Last length:    77	Average length: 22.33
Episode 60	Last length:    52	Average length: 24.56
Episode 70	Last length:    67	Average length: 28.63
Episode 80	Last length:   277	Average length: 41.01
Episode 90	Last length:   136	Average length: 84.41
Episode 100	Last length:    85	Average length: 84.43
Episode 110	Last length:   368	Average length: 90.95
Solved! Running reward is now 292.54680160988335 and the last episode runs to 9999 time steps!


## Actor-Critic Methods

In [9]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable

In [11]:
parser = {
    "gamma": 0.99,
    "seed": 543,
    "render": False,
    "log_interval": 10,
}
args = argparse.Namespace(**parser)

In [12]:
env = gym.make('CartPole-v0')
env.seed(args.seed)
torch.manual_seed(args.seed)

[2017-02-21 22:49:57,651] Making new env: CartPole-v0


In [13]:
SavedAction = namedtuple('SavedAction', ['action', 'value'])
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.action_head = nn.Linear(128, 2)
        self.value_head = nn.Linear(128, 1)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = self.action_head(x)
        state_values = self.value_head(x)
        return F.softmax(action_scores), state_values

In [14]:
model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)

In [15]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs, state_value = model(Variable(state))
    action = probs.multinomial()
    model.saved_actions.append(SavedAction(action, state_value))
    return action.data

def finish_episode():
    R = 0
    saved_actions = model.saved_actions
    value_loss = 0
    rewards = []
    for r in model.rewards[::-1]:
        R = r + args.gamma * R
        rewards.insert(0, R)
    rewards = torch.Tensor(rewards)
    rewards = (rewards - rewards.mean()) / rewards.std()
    for (action, value), r in zip(saved_actions, rewards):
        action.reinforce(r - value.data.squeeze())
        value_loss += F.smooth_l1_loss(value, Variable(torch.Tensor([r])))
    optimizer.zero_grad()
    final_nodes = [value_loss] + list(map(lambda p: p.action, saved_actions))
    gradients = [torch.ones(1)] + [None] * len(saved_actions)
    autograd.backward(final_nodes, gradients)
    optimizer.step()
    del model.rewards[:]
    del model.saved_actions[:]

In [16]:
running_reward = 10
for i_episode in count(1):
    state = env.reset()
    for t in range(10000): # Don't infinite loop while learning
        action = select_action(state)
        state, reward, done, _ = env.step(action[0,0])
        if args.render:
            env.render()
        model.rewards.append(reward)
        if done:
            break

    running_reward = running_reward * 0.99 + t * 0.01
    finish_episode()
    if i_episode % args.log_interval == 0:
        print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
            i_episode, t, running_reward))
    if running_reward > 200:
        print("Solved! Running reward is now {} and "
              "the last episode runs to {} time steps!".format(running_reward, t))
        break

Episode 10	Last length:   132	Average length: 14.91
Episode 20	Last length:    21	Average length: 15.53
Episode 30	Last length:    50	Average length: 16.97
Episode 40	Last length:   221	Average length: 22.16
Episode 50	Last length:    51	Average length: 30.68
Episode 60	Last length:    97	Average length: 34.25
Episode 70	Last length:   258	Average length: 111.09
Episode 80	Last length:    95	Average length: 109.82
Solved! Running reward is now 221.82367067917187 and the last episode runs to 9999 time steps!


# Deep Q-Network (DQN) with Experience Replay

See https://github.com/pytorch/tutorials/blob/master/Reinforcement%20(Q-)Learning%20with%20PyTorch.ipynb.